<a href="https://colab.research.google.com/github/Anurag-Singla/colab/blob/main/9_Prompt_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generative AI Prompt Engineering

In this lab we will use a famous Encoder-Decoder LLM: Flan-T5. You will first do simple tasks to get your hands dirty.

Then you will learn about few shot prompting, and see how at a certain point the LLM just cannot do the task.

You will finish by testing the different possible configurations.

## Install Required Dependencies

Now install the required packages to use Hugging Face transformers and datasets.

In [3]:
!pip install transformers~=4.0 'numpy<2' gensim nltk swifter

Load the datasets, Large Language Model (LLM), tokenizer, and configurator. Do not worry if you do not understand yet all of those components - they will be described and discussed later in the notebook.

In [1]:
from datasets import load_dataset
from transformers import TFAutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

## Doing Simple Tasks with Flan-T5

In this case we wil do simple sentiment analysis so you get the gist of how to use these LLMs. You will use the pre-trained Large Language Model (LLM) FLAN-T5 from Hugging Face. The list of available models in the Hugging Face `transformers` package can be found [here](https://huggingface.co/docs/transformers/index)

In [3]:
!pip install -U datasets huggingface_hub fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.3/515.3 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 21.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.33.2
    Uninstalling huggingface-hub-0.33.2:
      Successfully uninstalled huggingface-hub-0.33.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.

In [2]:
huggingface_dataset_name = "imdb"

dataset = load_dataset(huggingface_dataset_name)

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

Let's use from the train set, but it is the same for us now

In [4]:
import numpy as np
def get_random_review_and_label():
  random_index = np.random.randint(1, 25000)
  random_review = dataset['train'][random_index]['text']
  label = dataset['train'][random_index]['label']
  return random_review, label

random_review, label = get_random_review_and_label()

dash_line = '-'.join('' for x in range(100))

print(f'Review: \n\n{random_review}')
print(dash_line)
print(f'Label: {label}')

Review: 

Delightful film directed by some of the best directors in the industry today. The film is also casting some of the great actors of our time, not just from France but from everywhere.<br /><br />My favorite segments:<br /><br />14th arrondissement: Carol (Margo Martindale), from Denver, comes to Paris to learn French and also to make a sense of her life.<br /><br />Montmartre: there was probably not a better way to start this movie than with this segment on romantic Paris.<br /><br />Loin du 16ème: an image of Paris that we are better aware of since the riots in the Cités. Ana (Catalina Sandino Moreno) spends more time taking care of somebody else's kid (she's a nanny) than of her own.<br /><br />Quartier Latin: so much fun to see Gérard Depardieu as the "tenancier de bar" with Gena Rowlands and Ben Gazzara discussing their divorce.<br /><br />Tour Eiffel: don't tell me you didn't like those mimes!<br /><br />Tuileries: such a treat to see Steve Buscemi as the tourist who's ma

Let's now use the model! For that we need to use the Tokenizer to transform the text into the "model language" (more on this during the course). Also we need to download the model. Remember we are going to use a `TFAutoModelForSeq2SeqLM` model

In [5]:
model_name='google/flan-t5-base'

model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
/usr/local/lib/python3.11/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [6]:
sentence = random_review[:50]
print(f'Review trimmed: {sentence}')

sentence_encoded = tokenizer(sentence)

sentence_decoded = tokenizer.decode(
        sentence_encoded["input_ids"],
        skip_special_tokens=True
    )

print('ENCODED SENTENCE:')
print(sentence_encoded["input_ids"])
print('\nDECODED SENTENCE:')
print(sentence_decoded)

Review trimmed: Delightful film directed by some of the best direc
ENCODED SENTENCE:
[374, 2242, 1329, 814, 6640, 57, 128, 13, 8, 200, 5794, 75, 1]

DECODED SENTENCE:
Delightful film directed by some of the best direc


Now let's call the model. As this is a TFAutoModelForSeq2SeqLM this means that is a LLM for seq2seq tasks, like summarizing or text generation, so let's put our prompt that way.

In [7]:
import tensorflow as tf
review, label = get_random_review_and_label()

prompt = f"""
Analyze the sentiment of the following review:

{review}

Sentiment:

"""
print(prompt)
input = tokenizer(prompt)


Analyze the sentiment of the following review:

Diagnosis Murder is one of the only programs i watch regularly on TV now. The way all of the main characters have something to do in each of the episodes makes it so unlike today's shows where they concentrate on 1 or 2 people per episode and everyone else is shoved to the side. The way mark's brain works is also so obscure that you never really know what he is thinking, and if you think you do, you are wrong.<br /><br />Diagnosis Murder has tackled a diverse range of topics, not just in the cases but in each of the characters personal lives. Everything from racism and adoption, to terrorists and technology.<br /><br />As for it being for old people? I am 24!! I don't think i can be classed as old yet.<br /><br />I just want to know when they will be releasing all 8 seasons on DVD (not NTSC) so that i can watch them all in order. They seem to be doing it with lots of other programs from the same era so can they hurry it up a bit!

Sentim

In [8]:
model.generate(tf.constant([input['input_ids']]), max_new_tokens=50)

<tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[   0, 1465,    1]], dtype=int32)>

In [9]:
tokenizer.decode(
        model.generate(tf.constant([input['input_ids']]), max_new_tokens=50)[0],
        skip_special_tokens=True
    )

'positive'

And what was the real sentiment? Remember in this dataset `0` is negative and `1` is positive

In [10]:
label

1

## Summarize News without Prompt Engineering

In this use case, you will be generating a summary of news with Flan-T5.

Let's upload some simple dialogues from the dialogsum Hugging Face dataset. This dataset contains 10,000+ articles with the corresponding manually labeled summaries.

In [11]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

README.md: 0.00B [00:00, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

Print a couple of dialogues with their baseline summaries.

In [13]:
def get_random_dialogue_and_summary():
  # Implement this method

  random_index = np.random.randint(1, 10000)
  random_dialogue = dataset['train'][random_index]['dialogue']
  summary = dataset['train'][random_index]['summary']
  return random_dialogue, summary

random_dialogue, summary = get_random_dialogue_and_summary()

dash_line = '-'.join('' for x in range(100))

print(f'Dialogue: \n\n{random_dialogue}')
print(dash_line)
print(f'Summary: {summary}')

Dialogue: 

#Person1#: Get up as early as six o'clock only to be jammed at every crossroad and still late for work. What a life! I've had enough of it. 
#Person2#: Cool down, man. Everyone is fed up with the rush-hour traffic. But life isn't really all that. You should take the initiative and make some changes first. 
#Person1#: What should I do then? 
#Person2#: I recommend you ride a bike instead of commuting by bus. It may offer you many beneits. First, it's good for your health. I'm afraid it's not necessary for me to further elaborate. While lots of people spend time like an hour each morning exercising, a bike ride to work not only builds you up, but also makes full use of time. You might as well sleep out for a longer hour. 
#Person1#: I know cycling is always a more favorable choice than a bus. After all, it's a sport. But do you think it a pleasant experience to take in the dirty, pollued air on the road? 
#Person2#: Well, such things are just unavoidable in a great metropolis

Test the tokenizer encoding and decoding a simple sentence:

Now it's time to explore how well the base LLM summarizes a dialogue without any prompt engineering. **Prompt engineering** is an act of a human changing the **prompt** (input) to improve the response for a given task.

In [14]:
for i in range(3):
    dialogue, summary = get_random_dialogue_and_summary()
    inputs = tokenizer(dialogue)
    model_output = model.generate(tf.constant([inputs['input_ids']]), max_new_tokens=50)[0]
    output = tokenizer.decode(
        model_output,
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'Dialogue:\n{dialogue}')
    print(dash_line)
    print(f'Summary:\n{summary}')
    print(dash_line)
    print(f'Model Summary - Without prompt engineering:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
Dialogue:
#Person1#: You're not looking happy. What's the matter? 
#Person2#: Oh, nothing special. I'm just a bit tired. 
#Person1#: With the job? 
#Person2#: With everything, with everybody, with all this! 
#Person1#: A good suggestion for you. You need a holiday. 
#Person2#: It wasn't always like this, you know. 
#Person1#: What do you mean? 
#Person2#: Well, I mean. We always do the same thing. There's no variety in our lives. 
#Person1#: You need a holiday. That's what's the matter. 
#Person2#: Certainly, perhaps. 
---------------------------------------------------------------------------------------------------
Summary:
#Person2#'s tired of everything and #Person1# suggests #Person2# take a holiday.
--------------------------------------------------------------------------

You can see that the guesses of the model make some sense, but it doesn't seem to be sure what task it is supposed to accomplish. Seems it just makes up the next sentence in the dialogue. Prompt engineering can help here.

## Summarize Dialogue with an Instruction Prompt

Prompt engineering is an important concept in using foundation models for text generation.

<a name='3.1'></a>
### 3.1 - Zero Shot Inference with an Instruction Prompt

In order to instruct the model to perform a task - summarize a dialogue - you can take the dialogue and convert it into an instruction prompt. This is often called **zero shot inference**.  
Wrap the dialogue in a descriptive instruction and see how the generated text will change:

In [15]:
for i in range(3):
    dialogue, summary = get_random_dialogue_and_summary()
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
    """
    inputs = tokenizer(prompt)
    output = tokenizer.decode(
        model.generate(tf.constant([inputs['input_ids']]), max_new_tokens=50)[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'Dialogue:\n{dialogue}')
    print(dash_line)
    print(f'Summary:\n{summary}')
    print(dash_line)
    print(f'Model Summary - Zero shot inference prompt engineering:\n{output}\n')


---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
Dialogue:
#Person1#: Hello, Nancy. I'm sorry, but I just missed the 8:50 bus to the museum. I'm afraid I'll be a little late.
#Person2#: It doesn't matter. The next bus will be coming at 9:00. You can catch that one. The museum opens at 9:30.
---------------------------------------------------------------------------------------------------
Summary:
Nancy tells #Person1# that #Person1# won't be late if #Person1# catches the next bus.
---------------------------------------------------------------------------------------------------
Model Summary - Zero shot inference prompt engineering:
The next bus will be coming at 9:00.

---------------------------------------------------------------------------------------------------
Example  2
----------------------------------------------

This is much better! But the model still does not pick up on the nuance of the conversations though.

## Summarize Dialogue with One Shot and Few Shot Inference

**One shot and few shot inference** are the practices of providing an LLM with either one or more full examples of prompt-response pairs that match your task - before your actual prompt that you want completed. This is called "in-context learning" and puts your model into a state that understands your specific task.  

## One Shot Inference



In [16]:
def make_prompt_and_return_real_summary(number_of_shots):
    prompt = ''
    for i in range(number_of_shots):
        dialogue, summary = get_random_dialogue_and_summary()

        # The stop sequence '{summary}\n\n\n' is important for FLAN-T5. Other models may have their own preferred stop sequence.
        prompt += f"""
Dialogue:

{dialogue}

Summary:

{summary}


"""

    dialogue_to_analise , real_summary = get_random_dialogue_and_summary()

    prompt += f"""
Dialogue:

{dialogue_to_analise}

Summary:

"""

    return prompt, real_summary

Construct the prompt to perform one shot inference:

In [17]:
one_shot_prompt, real_summary = make_prompt_and_return_real_summary(1)

print(one_shot_prompt)


Dialogue:

#Person1#: Could we borrow the company van for a fundraiser this weekend?
#Person2#: That would be a possibility. Where is this fundraiser taking place?
#Person1#: It is in the hotel ballroom down the street.
#Person2#: Do you need it for the whole weekend?
#Person1#: We will need it for both days.
#Person2#: We will need to know who will be driving the van.
#Person1#: The van will be driven by Mary and me.
#Person2#: It needs to be back on Sunday night. Can you arrange for that?
#Person1#: Oh yeah, no problem. Would you mind if we borrowed a few of the chairs from the lunchroom.
#Person2#: Just keep track of everything and get it all back where you took it from by Sunday evening.

Summary:

#Person1# wants to borrow the company van for a fundraiser and also asks for some chairs. #Person2# agrees but asks #Person1# to bring them back by Sunday evening.



Dialogue:

#Person1#: They say you've got a job in the New York City.
#Person2#: Yeah, we say it the United Nations.
#Pe

Now pass this prompt to perform the one shot inference:

In [18]:
for i in range (3):
  one_shot_prompt, real_summary = make_prompt_and_return_real_summary(1)
  inputs = tokenizer(one_shot_prompt)
  output = None # Get output
  print(dash_line)
  print(f'Example {i + 1}')
  print(dash_line)
  print(f'Dialogue:\n{one_shot_prompt}')
  print(dash_line)
  print(f'Summary:\n{real_summary}')
  print(dash_line)
  print(f'Model Summary - One shot inference prompt engineering:\n{output}\n')

Token indices sequence length is longer than the specified maximum sequence length for this model (667 > 512). Running this sequence through the model will result in indexing errors


---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
Dialogue:

Dialogue:

#Person1#: During the last two years, there has been a great increase in goods stolen abroad.
#Person2#: If they steal the entire package, that would be theft. So they don't do that. Generally, thieves open the case and take part of the contents out. Then fill the case so that the gross weight will be the same.
#Person1#: If the goods are received in apparent good order and condition, the steamship company doesn't have liability for pilferage. How do we protect ourselves?
#Person2#: The insurance policy protects us.
#Person1#: Is it true that products of high value such as watches, jewels and luxury clothing items are often subject to pilferage?
#Person2#: It's absolutely true in the United States that losses from pilferage have been limited to high valued g

### Few Shot Inference

Let's explore few shot inference by adding two more full dialogue-summary pairs to your prompt.

In [19]:
for i in range (3):
  few_shot_prompt, real_summary = make_prompt_and_return_real_summary(5)
  inputs = tokenizer(few_shot_prompt)
  output = tokenizer.decode(
      model.generate(tf.constant([inputs['input_ids']]), max_new_tokens=50)[0],
      skip_special_tokens=True
  )

  print(dash_line)
  print(f'Example {i + 1}')
  print(dash_line)
  print(f'Dialogue:\n{few_shot_prompt}')
  print(dash_line)
  print(f'Summary:\n{real_summary}')
  print(dash_line)
  print(f'Model Summary - Few shot inference prompt engineering:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
Dialogue:

Dialogue:

#Person1#: Hello sir, how may I help you? 
#Person2#: I would like to buy some flowers, please. Something really nice. 
#Person1#: I see, may I ask what the occasion is? 
#Person2#: It's not really an occasion, it's more like I'm sorry. 
#Person1#: Very well. This arrangement here is very popular among regretful husbands and boyfriends. It has a dozen long stem red roses with a couple of sunflowers and a single orchid that stands out. It includes a small teddy bear to achieve the effect of immediate forgiveness. 
#Person2#: I think I'm gonna need more than just a dozen red roses and a bear. What else do you recommend? 
#Person1#: Mmm, well this is our I'm sorry I cheated on you package. Two dozen red roses lined with tulips, carnations and lilies. The fragra

In this case, few shot did not provide much of an improvement over one shot inference.  And, anything above 5 or 6 shot will typically not help much, either.  Also, you need to make sure that you do not exceed the model's input-context length which, in our case, if 512 tokens.  Anything above the context length will be ignored.

However, you can see that feeding in at least one full example (one shot) provides the model with more information and qualitatively improves the summary overall.

## Configuration Parameters

In [20]:
generation_config = GenerationConfig(max_new_tokens=100, do_sample=True, temperature=2.0)

inputs = tokenizer(few_shot_prompt)
output = tokenizer.decode(
    model.generate(tf.constant([inputs['input_ids']]), generation_config=generation_config)[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{real_summary}\n')

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
They are serving dinner. Here does the menu from their grocery lists. In this article Person One asks for healthy options and gives tips.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# prepares dinner with only vegetables without #Person2#'s favorite bread or dessert because #Person1# thinks greens can improve metabolism.



Comments related to the choice of the parameters in the code cell above:
- Choosing `max_new_tokens=10` will make the output text too short, so the dialogue summary will be cut.
- Putting `do_sample = True` and changing the temperature value you get more flexibility in the output.

As you can see, prompt engineering can take you a long way for this use case, but there are some limitations.